In [7]:
import torch
import torch.nn as nn
import math

# LayerNorm



## 核心思想

对每个样本（token）自己的所有特征独立做“均值为 0、标准差为 1”的归一化，完全不依赖 batch 大小。

![LN](pic/LN.png)

## 为什么这么做

由于对每个样本做了归一化，首先带来的好处是梯度是稳定的，不会过大或者过小，而梯度稳定则是训练稳定的重要前提。

In [12]:

class layernorm(nn.Module):
    def __init__(self, input_dim, eps=1e-5) -> None:
        super().__init__()
        self.eps = eps
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.ones(self.input_dim))
        self.bias = nn.Parameter(torch.zeros(self.input_dim))

    def forward(self, x):
        mean = x.mean(dim= -1, keepdim=True)
        var = x.var(dim= -1, keepdim=True, unbiased=False)
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight * x_norm + self.bias

## 代码测试

In [13]:
x = torch.tensor([[1., 2., 3., 4.], [10., 20., 30., 40.]], dtype=torch.float32)
ln = layernorm(4)
x_norm = ln(x)
x_norm

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]], grad_fn=<AddBackward0>)

## 一些值得注意的

### `nn.Parameter()`的用法

* `nn.Parameter()`用于自定义可学习的参数，是tensor的子类。

    当神经网络继承自nn.Module时，如果其中存在一些参数是需要纳入模型本身的参数里参与反向传播过程的，则可以使用该方法创建参数。

* 创建方法即为使用`nn.Parameter()`方法包裹tensor:

    `self.weight = nn.Parameter(torch.ones(self.input_dim))`


# RMS Norm



## 核心思想

![LN](pic/RMS_Norm.png)

RMSNorm 主要是在 LayerNorm 的基础上去掉了减均值这一项，其计算效率更高且没有降低性能。

因为原论文中指出LayerNorm发挥作用主要是其缩放不变性（即除以标准差），而不是平移不变性（即减去均值）


In [15]:
class RMS_Norm(nn.Module):
    def __init__(self, input_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.input_dim = input_dim
        self.weight = nn.Parameter(torch.ones(self.input_dim))
    def forward(self, x):
        rms = torch.sqrt(x.pow(2).mean(dim=-1, keepdim=True) + self.eps)
        return x * self.weight / rms


## 代码测试

In [ ]:
x = torch.tensor([[1., 2., 3., 4.], [10., 20., 30., 40.]], dtype=torch.float32)
rmsn = RMS_Norm(4)
x_norm = ln(x)
x_norm

tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.3416, -0.4472,  0.4472,  1.3416]], grad_fn=<AddBackward0>)

# Batch Norm

# MLP


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim) -> None:
        super().__init__()
        self.input_dim = input_dim
        self.w1 = nn.Linear(input_dim, input_dim * 4)
        self.relu = nn.ReLU()
        self.w2 = nn.Linear(input_dim * 4, input_dim)
    def forward(self, x):
        return self.w2(self.relu(self.w1(x)))


## 代码测试

In [ ]:
x = torch.rand(2, 3, 24)
mlp = MLP(24)
x = mlp(x)
print("x.shape:", x.shape)
x

# SwiGLU

## 常见的激活函数
* ReLU

    $ReLU(x) = \max(0, x)$

* GeLU

    论文《Gaussian Error Linear Units（GELUs）》提出了GELU，GeLU激活函数是处处可微的非线性函数，比ReLU更平滑

* Swish

    论文《Swish: a Self-Gated Activation Function》提出了Swish，这也是对带有非零负值梯度的ReLU平滑版本，同样是个处处可微的非线性函数，且有一个参数beta用于控制函数的形状。

* Silu

    beta=1的swish函数

![comparation](pic/af.png)

## GLU

GLU（Gated Linear Units）其实不算是一种激活函数，而是一种神经网络层。它是一个线性变换后面接门控机制的结构。其中门控机制是一个sigmoid函数用来控制信息能够通过多少。

$GLU(x,W,V,b,c) = \sigma (xW + b) \odot (xV + c)$

这里$\sigma$代表SIGMOD函数，$\odot$代表逐元素乘。

## SwiGLU

把上面公式的激活函数换成swish函数：

$GLU(x,W,V,b,c) = Swish_1 (xW + b) \odot (xV + c)$

就是SwiGLU



In [22]:
class SwiGLU(nn.Module):
    def __init__(self, input_dim,  multiple_of=64) -> None:
        super().__init__()
        self.input_dim = input_dim

        self.hidden_dim = 4 * self.input_dim
        self.hidden_dim = int(2 * self.hidden_dim / 3)
        self.hidden_dim = multiple_of * ((self.hidden_dim + multiple_of - 1) // multiple_of)

        self.w1 = nn.Linear(self.input_dim, self.hidden_dim)
        self.w2 = nn.Linear(self.hidden_dim, self.input_dim)
        self.w3 = nn.Linear(self.input_dim, self.hidden_dim)
        self.silu = nn.SiLU()
    
    def forward(self, x):
        return self.w2(self.silu(self.w1(x)) * self.w3(x))



## 代码测试

In [ ]:
x = torch.rand(2, 3, 24)
mlp = SwiGLU(24)
x = mlp(x)
print("x.shape:", x.shape)
x

## 值得一提的是

### 为什么要重新计算隐藏层维度

传统FFN使用4倍的input_dim作为hidden_dim（见上述MLP代码），此时不妨do the math：

* 传统FFN使用了两个线性层，参数量为 $ dim \times 4dim \times 2$
* SwiGLU使用了三个线性层，参数量为 $ dim \times hidden \_ dim  \times 3$
* 想要保证传统FFN与SwiGLU参数量上不发生太大变化，那么 $ hidden \_ dim $选择为 $ \cfrac{2}{3} \times 4dim $ 是最好的
* 然而NVIDIA GPU 的 Tensor Core 要求矩阵维度是 8/16/32/256 的倍数才能发挥最大算力，因此还得将$ hidden \_ dim $取值为8/16/32/256 的倍数，同时接近$ \cfrac{2}{3} \times 4dim $

### 取整技巧

其中

        self.hidden_dim = multiple_of * ((self.hidden_dim + multiple_of - 1) // multiple_of)

这个操作是将self.hidden_dim重新取值为可以被multiple_of整除的数，数学表达为：

$ceil(x / m) = (x + m - 1) // m$

* **情况1**: x 恰好是 m 的倍数，设 x = k*m

    (k*m + m - 1) // m = k + (m-1)//m = k + 0 = k  ✓

* **情况2**: x 有余数，设 x = k*m + r (0 < r < m)

    (k*m + r + m - 1) // m = k + (r + m - 1)//m = k + 1  ✓


# DecoderLayer

# EncoderLayer